# Install needed libraries

In [ ]:
!git clone https://github.com/aub-mind/arabert.git
!pip install simpletransformers

Cloning into 'arabert'...
remote: Enumerating objects: 564, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 564 (delta 25), reused 22 (delta 22), pack-reused 535
Receiving objects: 100% (564/564), 9.11 MiB | 21.06 MiB/s, done.
Resolving deltas: 100% (326/326), done.
     |████████████████████████████████| 249 kB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 46.6 MB/s 
     |████████████████████████████████| 1.8 MB 36.8 MB/s 
     |████████████████████████████████| 1.2 MB 43.8 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 342 kB 43.8 MB/s 
     |████████████████████████████████| 4.2 MB 36.1 MB/s 
     |████████████████████████████████| 10.1 MB 34.2 MB/s 
     |████████████████████████████████| 84 kB 2.8 MB/s 
     |████████████████████████████████| 596 kB 39.9 MB/s 
     |████████████████████████████████| 145 kB 30.5 MB/s 
     |████████████████████████████

     |████████████████████████████████| 126 kB 5.2 MB/s 


In [ ]:
!pip install simpletransformers

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
import torch
import json, argparse
from collections import Counter
import re
import string
import logging
import os
import numpy as np
import pandas as pd

# Show the preprocessing

In [ ]:
from arabert.preprocess import ArabertPreprocessor


model_name = 'aubmindlab/bert-base-arabertv02'
model_type = 'bert'

arabert_prep = ArabertPreprocessor(model_name=model_name)

text = "ولن نبالغَ إذا قلنا إنّ الهاتفَ أو كمبيوترً المكتب في زَمَنِنا هذا ضروريٌ"
arabert_prep.preprocess(text)

'ولن نبالغ إذا قلنا إن الهاتف أو كمبيوتر المكتب في زمننا هذا ضروري'

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/381 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/805k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.52M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
tokenizer("ولن نبالغ إذا قلنا إن هاتف أو كمبيوترً المكتب في زمننا هذا ضروري")

{'input_ids': [2, 3387, 3130, 2479, 1217, 18501, 476, 6846, 440, 1, 3937, 305, 3777, 325, 434, 17039, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# Checking the Google Colab using cuda instead of cpu.

if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## Load data sets

In [ ]:
# Load our dataset

def load(file_name):
  with open(file_name) as f:
    dataset = json.load(f)['data']

  data = []
  for article in dataset:
    for paragraph in article['paragraphs']:
      data.append(paragraph)

  return data

In [ ]:
train1 = load('train.json')

test = load('test.json')

In [ ]:
# Load AQAD dataset

import requests
url = 'https://raw.githubusercontent.com/adelmeleka/AQAD/master/AQQAD%201.0/FINAL_AAQAD-v1.0.json'
resp = requests.get(url)
dataset = json.loads(resp.text)['data']
train2 = []
for article in dataset:
  for paragraph in article['paragraphs']:
    for qa in paragraph['qas']:
      if not qa["is_impossible"]:
        train2.append(paragraph)
print(train2[0])

{'qas': [{'question': 'متى بدأت بيونسي تصبح شعبية؟', 'id': 1, 'answers': [{'text': 'في أواخر التسعينات', 'answer_start': 220}], 'is_impossible': False}, {'question': 'ما هي المجالات التي تنافس عليها بيونسيه عندما كانت تكبر؟', 'id': 2, 'answers': [{'text': 'غناء ورقص', 'answer_start': 173}], 'is_impossible': False}, {'question': 'متى غادرت بيونسي طفل القدر وتصبح مغنية منفردة؟', 'id': 3, 'answers': [{'text': '(2003)،', 'answer_start': 515}], 'is_impossible': False}, {'question': 'في أي مدينة و ولاية نشأت بيونسيه؟', 'id': 4, 'answers': [{'text': 'هيوستن بولاية تكساس،', 'answer_start': 88}], 'is_impossible': False}, {'question': 'في أي عقد أصبحت بيونسي مشهورة؟', 'id': 5, 'answers': [{'text': 'أواخر التسعينات', 'answer_start': 223}], 'is_impossible': False}, {'question': 'في أي مجموعة R&B كانت هي المغنية الرئيسية؟', 'id': 6, 'answers': [{'text': 'دستنيز تشايلد', 'answer_start': 290}], 'is_impossible': False}, {'question': 'ما الألبوم الذي جعلها فنانة معروفة عالميا؟', 'id': 7, 'answers': [{'

In [ ]:
print("length of our train data:", len(train1))
print("length of AQAD dataset :", len(train2))


print("take 2800 samples from AQAD dataset")
training_data = train1 + train2[:2800]

print("length of the combined dataset:", len(training_data))

length of our train data: 222
length of AQAD dataset : 11493
take 2800 samples from AQAD dataset
length of the combined dataset: 3022


In [ ]:
training_data[500]

{'context': 'بدءًا بالاستخدام المتزايد للفحم الذي تزامن مع الثورة الصناعية، تحول استهلاك الطاقة بشكل ثابت من الخشب والكتل الحيوية إلى الوقود الحفري. ونتج التطور المبكر لتقنيات استخدام الطاقة الشمسية، والذي بدأ في ستينيات القرن التاسع عشر، عن توقع احتمالية ندرة الفحم في وقت قريب. ومع ذلك، فقد أصبح تطور تقنيات استخدام الطاقة الشمسية أبطء في بدايات القرن العشرين نظرًا لازدياد استخدام الفحم والبترول ولوفرته ورخص ثمنه.\nأدى حظر استخدام النفط في عام 1973 وأزمة الطاقة التي حدثت في عام 1979 إلى إعادة تنظيم سياسات استهلاك الطاقة حول العالم وإعادة الاهتمام مجددًا بتطوير تقنيات استخدام الطاقة الشمسية.. وقد ركزت استراتيجيات توزيع الطاقة على البرامج المحفزة مثل برنامج "استخدام الطاقة الفولتوضوئية الفيدرالي" في الولايات المتحدة الأمريكية وبرنامج "صن شاين" في اليابان. كذلك، ومن مظاهر الجهود التي بذلت أيضًا إنشاء أماكن ومعامل للبحث العلمي في الولايات المتحدة الأمريكية (معامل SERI والمعروفة حاليًا بالمعامل القومية لمصادر الطاقة المتجددة وفي اليابان؛ (منظمة تطوير تكنولوجيا الصناعة والطاقة الجديدة)وفي أل

In [ ]:
# Split train data

import sklearn
from sklearn.model_selection import train_test_split

train_data, eval_data = train_test_split(training_data)

# Training

In [ ]:
# Configure the model

models_args = QuestionAnsweringArgs()
models_args.train_batch_size = 2
models_args.evaluate_during_training = True
models_args.n_best_size = 3
models_args.save_model_every_epoch = False
models_args.save_steps = -1
models_args.num_train_epochs = 2
models_args.manual_seed = 109
models_args.output_dir = "/content/output/"

In [ ]:
### Remove output folder (useful if you retrain)

!rm -rf outputs
!rm -rf output

In [ ]:
# Instatiate the model and set the "args" to models_args variable

model = QuestionAnsweringModel(model_type, model_name, args= models_args)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized fr

In [ ]:
# training the model

model.train_model(train_data=train_data, eval_data = eval_data, acc=sklearn.metrics.accuracy_score)

convert squad examples to features:   0%|          | 1/11393 [00:41<131:29:11, 41.55s/it]Could not find answer: 'بعضهما بروابط رخوة من قاعدتي البيورين و البيريميدين ؛ و لكن من الملاحظ أن :' vs. 'بعضهما بروابط رخوة من قاعدتي البيورين و البيريميدين ؛ و لكن من الملاحظ أن :'
Could not find answer: 'بعضهما بروابط رخوة من قاعدتي البيورين و البيريميدين ؛ و لكن من الملاحظ أن :' vs. 'بعضهما بروابط رخوة من قاعدتي البيورين و البيريميدين ؛ و لكن من الملاحظ أن :'
Could not find answer: 'بعضهما بروابط رخوة من قاعدتي البيورين و البيريميدين ؛ و لكن من الملاحظ أن :' vs. 'بعضهما بروابط رخوة من قاعدتي البيورين و البيريميدين ؛ و لكن من الملاحظ أن :'
add example index and unique id: 100%|██████████| 11393/11393 [00:00<00:00, 671345.56it/s]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to d

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/5823 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


convert squad examples to features:   0%|          | 0/3701 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/3701 [00:13<14:11:35, 13.81s/it]

convert squad examples to features: 100%|██████████| 3701/3701 [00:26<00:00, 137.57it/s]


add example index and unique id: 100%|██████████| 3701/3701 [00:00<00:00, 655730.96it/s]


Running Evaluation:   0%|          | 0/482 [00:00<?, ?it/s]

...



convert squad examples to features:   0%|          | 0/3701 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/3701 [00:13<13:49:35, 13.45s/it]

convert squad examples to features: 100%|██████████| 3701/3701 [00:26<00:00, 141.41it/s]


add example index and unique id: 100%|██████████| 3701/3701 [00:00<00:00, 455183.39it/s]


Running Evaluation:   0%|          | 0/482 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 3701/3701 [00:27<00:00, 135.59it/s]

add example index and unique id: 100%|██████████| 3701/3701 [00:00<00:00, 573401.27it/s]


Running Evaluation:   0%|          | 0/482 [00:00<?, ?it/s]

(11646,
 {'acc': [0.38914910226385635,
   0.4953161592505855,
   0.5761124121779859,
   0.5858704137392662,
   0.637392661982826,
   0.7029664324746292,
   0.7283372365339579],
  'correct': [997, 1269, 1476, 1501, 1633, 1801, 1866],
  'eval_loss': [-4.5923114950726145,
   -6.530358532157677,
   -8.437038057572614,
   -8.929185036307054,
   -9.678909491701244,
   -9.976594917012449,
   -10.456107365145229],
  'global_step': [2000, 4000, 5823, 6000, 8000, 10000, 11646],
  'incorrect': [428, 392, 268, 269, 202, 148, 132],
  'similar': [1137, 901, 818, 792, 727, 613, 564],
  'train_loss': [5.4306640625,
   6.8095703125,
   1.0446064472198486,
   1.952789306640625,
   0.02742934226989746,
   4.094839096069336e-05,
   1.3053417205810547e-05]})

What does "Could not find answer" mean?
<br/> 


This means that the script that converts the examples to features can't find the answers it should be finding. Rather than trying to predict those, it ignores them.

This means that these examples won't be used for training, reducing the total number of examples that will be used. If it is a small portion of the total number of examples, it shouldn't impact the resulting accuracy much. If it is a significant portion of the examples then it would be a good idea to look into it to see if there's a quick fix.

In [ ]:
# evaluate on test set

result, predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score )

add example index and unique id: 100%|██████████| 639/639 [00:00<00:00, 287941.58it/s]


Running Evaluation:   0%|          | 0/81 [00:00<?, ?it/s]

In [ ]:
result

{'acc': 0.20344287949921752,
 'correct': 130,
 'eval_loss': -10.27758487654321,
 'incorrect': 171,
 'similar': 338}

In [ ]:
predictions["incorrect_text"]

{'106366036897': {'predicted': 'الشعر',
  'question': 'ما هو سر محبته للجمال و الالوان وبالذات اللون الاخضر؟',
  'truth': 'منزلهم الدمشقي كان لديهم أغلب أصناف الزروع الشاميّة من زنبق وريحان وياسمين ونعناع ونارنج.'},
 '112079412992': {'predicted': 'من قبيلة كندة',
  'question': 'ما جنسية جندح بن حجر؟',
  'truth': 'هو شاعرعربي'},
 '114069415149': {'predicted': 'حوالي 400 كيلومتر في الاتجاه الجنوبي الغربي،',
  'question': 'كم تبعد مكة المكرمة عن الطائف؟',
  'truth': 'حوالي 120 كيلومترا في الاتجاه الشرقي،'},
 '118072267433': {'predicted': '1916م',
  'question': 'في أي عام إنتقلت أم كلثوم و والدها إلى القاهرة؟',
  'truth': '1922م.'},
 '118785899203': {'predicted': 'تلبس العقال وملابس',
  'question': 'ماذا كانت أم كلثوم تلبس في الحفلات عندما كانت صغيرة؟',
  'truth': 'العقال وملابس الأولاد.'},
 '122985932183': {'predicted': '30%',
  'question': 'ما نسبة اصاية الاولاد بالتوحد؟',
  'truth': '4 مرات أكثر من البنات.'},
 '13338187135': {'predicted': 'في القرآن،',
  'question': 'ما هو الاعجاز العلم

In [ ]:
predictions["similar_text"]

{'100269981551': {'predicted': '9',
  'question': 'كم عدد البطولات اللتى حققها زيدان مع النادى الملكى؟',
  'truth': '9 بطولات في عامين ونصف'},
 '101765027998': {'predicted': 'دابّةً',
  'question': 'ماذا رأى الرسول حين أخرجهجبريل إلى باب المسجد؟',
  'truth': 'دابّةً بيضاء بين البغل والحمار،'},
 '102861223894': {'predicted': '1919 إلى 1945',
  'question': 'متى تم إنشاء منظمة عصبة الأمم؟',
  'truth': '1919'},
 '104499435818': {'predicted': '1505',
  'question': 'متي تم اكتشاف موريشيوس؟',
  'truth': 'في العام 1505'},
 '105236247076': {'predicted': 'أكتوبر 2009',
  'question': 'متى عينت سفيرة للنوايا الحسنة؟',
  'truth': 'وفي أكتوبر 2009'},
 '105673018704': {'predicted': 'حرب أكتوبر "حرب العاشر من رمضان" كما تعرف في مصر أو حرب تشرين التحريرية كما تعرف في سوريا أو حرب يوم الغفران (بالعبرية: מלחמת יום כיפור، ميلخمت يوم كيبور) كما تعرف في إسرائيل، هي حرب شنتها كل من مصر وسوريا على إسرائيل',
  'question': 'بماذا تسمي حرب اكتوبر في اسرائيل؟',
  'truth': 'حرب يوم الغفران'},
 '109564956217': {'pr

In [ ]:
predictions["correct_text"]

{'10737648493': 'الاتحاد الدولي لكرة القدم (الـفيفا).',
 '121591221310': 'بالتشبه بأحمد زكي وتقليده،',
 '127784589749': '4,675,710 ملايين نسمة.',
 '128289817127': 'رئيس المخابرات الحربية والاستطلاع.',
 '13675524569': 'محمد بن عبد الله بن عبد المطلب،',
 '138768012694': 'في القرن السادس الهجري',
 '145022408477': 'ثلاثمائة وبضعة عشر رجلاً،',
 '153159225042': 'Sultan Süleyman-ı Evvel)',
 '156381511284': '20%',
 '165262912649': 'في بلدة محرونة في قضاء صور جنوب لبنان',
 '174456986650': 'مارس 1965',
 '180300657120': 'ثورات وفتناً كثيرة،',
 '188260747433': 'الخطيب التبريزي',
 '195740200624': 'نظام ملكي دستوري',
 '201991055920': 'الخليفة عمر بن الخطاب',
 '208278542813': 'حارس المرمى',
 '212974754962': 'الصادق المهدي',
 '223790039437': 'Emilio Estefan',
 '228310460636': '336 هـ،',
 '230223442879': 'كلية عسكرية،',
 '235375779884': 'في صباح اليوم التّالي',
 '23705453527': '6 أبريل 1974،',
 '239159489684': '2014',
 '241639333474': '(19 نوفمبر 1954)،',
 '242393780356': 'لأكثر من 25 عاما',
 '25545769

# Evaluate 

In [ ]:
predictions, raw_outputs = model.predict(test)

add example index and unique id: 100%|██████████| 639/639 [00:00<00:00, 356593.97it/s]


Running Prediction:   0%|          | 0/81 [00:00<?, ?it/s]

Obtain metrics EM and F1 on our data set. This follows the **modified** offical SQuAD 1.1 evaluation script

In [ ]:
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys
import nltk
import random
nltk.download('punkt')
from random import randint

test = load('test.json')
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    
    def remove_articles_ar(text):
        return re.sub('\sال^|ال', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles_ar(remove_articles(remove_punc(lower(s)))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))

def metric_max_over_ground_truths(metric_fn, predictions, ground_truth):
    scores_for_ground_truths = []
    for prediction in predictions:
        score = metric_fn(prediction, ground_truth)
        print("prediction:", prediction, "   --score: ", score)        
        scores_for_ground_truths.append(score)
    
    return max(scores_for_ground_truths)


def evaluate(dataset, predictions):
    f1 = exact_match = total = exact_sentence = inclusion = random = 0
    for article in dataset:
        print("context: ",article["context"])
        for qa in article['qas']:
            total += 1
            print("question: ",qa["question"])
            
            ground_truth = {"id": qa["id"],'answer':[qa['answers'][0]["text"]]}
            answer_gt = ground_truth["answer"][0]
            prediction = next(item for item in predictions if item["id"] == ground_truth["id"])
            answers_pred = prediction["answer"]
            
            if answers_pred == "empty": answers_pred = " "

            print("\n","gt:", answer_gt)

            exact_match += metric_max_over_ground_truths(exact_match_score, answers_pred, answer_gt)
            f1 += metric_max_over_ground_truths(f1_score, answers_pred, answer_gt)
            print("------------------ \n")
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
evaluate(test, predictions)

Streaming output truncated to the last 5000 lines.
prediction: 1710    --score:  0
------------------ 

question:  متي أعلنت موريشيوس استقلالها؟

 gt: 12 مارس 1968،
prediction:     --score:  False
prediction:     --score:  0
------------------ 

context:  في هذه الأيام، تُعرف موريشيوس على أنها جمهورية ديموقراطية مبنية على نموذج ويستمينستير والذي يضمن الفصل بين كل من القوى التشريعية والتنفيذية والقضائية. تتمتع الجزيرة الآن باستقرار سياسي حيث يتم انتخاب 62 عضواً للجمعية الوطنية كل 5 أعوام. وفي حين وجود رئيس للدولة، فرئيس الوزراء هو من يحمل القوى التنفيذية بالإضافة إلى قيادة الحكومة.
question:  كم يبلغ أعضاء الجمعية الوطنية في موريشيوس؟

 gt: 62 عضواً
prediction: 62    --score:  False
prediction: 62 عضواً    --score:  True
prediction: انتخاب 62    --score:  False
prediction: 62    --score:  0.6666666666666666
prediction: 62 عضواً    --score:  1.0
prediction: انتخاب 62    --score:  0.5
------------------ 

question:  متي يتم انتخاب أعضاءالجمعية الوطنية في موريشيوس؟

 gt: كل 5 أعوام.
predic

{'exact_match': 38.65414710485133, 'f1': 63.66639606153087}

# Predict

In [ ]:
# Load the model 

from simpletransformers.question_answering import QuestionAnsweringModel,QuestionAnsweringArgs

model_name = '/content/outputs/best_model'
model_type = 'bert'

with open("/content/outputs/best_model/tokenizer_config.json") as f:
  model_args = json.load(f)
 

model = QuestionAnsweringModel(model_type, model_name, args= model_args)


In [ ]:
to_predict = [
    {
        "context": 'تقع سويسرا في قلب القارة الأوروبية وتحيط بها خمس دول، وهي ألمانيا من الشمال وإيطاليا من الجنوب والنمسا وإمارة ليختنشتاين من الشرق وفرنسا من الغرب، وليست لها منافذ بحرية وتبلغ مساحتها حوالي 41300 كيلومترا مربعا. وتتكون سويسرا من ثلاث مناطق جغرافية وهي: سلسلة جبال الألب، التي تمتد في الجنوب وتغطي حوالي ثلثي مساحة البلاد ويبلغ ارتفاع أعلى قممها "بونتا دوفور" 4638م. ثم هناك سلسلة جبال جورا والتي تمتد على شكل هلال في غرب وشمال البلاد، وتمثل الحد الفاصل بين سويسرا وفرنسا وتغطي نحو 12٪ من المساحة الكلية، ويبلغ ارتفاع أعلى قممها "كريت دو لا نيج" 1718 م، وبين هاتين المجموعتين من السلاسل الجبلية، تمتد منطقة الهضبة السهلية التي تضم معظم المدن والقرى السويسرية.',
        "qas": [
            {
                "question": "كم دولة على الحدود مع سويسرا",
                "id": "0",
            },
            {
                "question": "كم دولة تحيط بسويسرا؟",
                "id": "1",
            },
            {
                "question": "ما مساحتها",
                "id": "2",
            },
            {
                "question": "ما هي المساحة الاجمالية لسويسرا؟",
                "id": "3",
            },
            {
                "question": "أي دولة تقع جنوب سويسرا؟",
                "id": "4",
            },
            {
                "question": "ما هي الدول الواقعة شرق سويسرا؟",
                "id": "5",
            },

        ],
    }
]

In [ ]:
GT = [{
 "context": 'تقع سويسرا في قلب القارة الأوروبية وتحيط بها خمس دول، وهي ألمانيا من الشمال وإيطاليا من الجنوب والنمسا وإمارة ليختنشتاين من الشرق وفرنسا من الغرب، وليست لها منافذ بحرية وتبلغ مساحتها حوالي 41300 كيلومترا مربعا. وتتكون سويسرا من ثلاث مناطق جغرافية وهي: سلسلة جبال الألب، التي تمتد في الجنوب وتغطي حوالي ثلثي مساحة البلاد ويبلغ ارتفاع أعلى قممها "بونتا دوفور" 4638م. ثم هناك سلسلة جبال جورا والتي تمتد على شكل هلال في غرب وشمال البلاد، وتمثل الحد الفاصل بين سويسرا وفرنسا وتغطي نحو 12٪ من المساحة الكلية، ويبلغ ارتفاع أعلى قممها "كريت دو لا نيج" 1718 م، وبين هاتين المجموعتين من السلاسل الجبلية، تمتد منطقة الهضبة السهلية التي تضم معظم المدن والقرى السويسرية.',
 'qas': [
  {'answers': [{'text': 'خمس دول'}],
   'id': '0',
   'question': 'كم دولة على الحدود مع سويسرا'},
  {'answers': [{'text': 'خمس دول'}],
   'id': '1',
   'question': 'كم دولة تحيط بسويسرا؟'},
  {'answers': [{'text': 'حوالي 41300 كيلومترا مربعا'}],
   'id': '2',
   'question': 'ما مساحتها'},
   {'answers': [{'text': 'حوالي 41300 كيلومترا مربعا'}],
   'id': '3',
   'question': 'ما هي المساحة الاجمالية لسويسرا؟'},
   {'answers': [{'text': 'إيطاليا'}],
   'id': '4',
   'question': 'أي دولة تقع جنوب سويسرا؟'},
   {'answers': [{'text': 'النمسا وإمارة ليختنشتاين'}],
   'id': '5',
   'question': 'ما هي الدول الواقعة شرق سويسرا؟'},
   ]
   
   }]

In [ ]:
prediction, raw_outputs = model.predict(to_predict)

add example index and unique id: 100%|██████████| 6/6 [00:00<00:00, 45425.68it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction

[{'answer': ['سويسرا وفرنسا',
   'وفرنسا',
   'سويسرا وفرنسا وتغطي نحو 12٪ من المساحة الكلية، ويبلغ ارتفاع أعلى قممها "كريت دو لا'],
  'id': '0'},
 {'answer': ['خمس', 'خمس دول،', 'خمس دول'], 'id': '1'},
 {'answer': ['', 'حوالي 41300 كيلومترا مربعا.'], 'id': '2'},
 {'answer': ['حوالي 41300 كيلومترا مربعا.',
   'حوالي 41300 كيلومترا مربعا',
   'حوالي 41300 كيلومترا مربعا. وتتكون سويسرا من ثلاث مناطق جغرافية وهي: سلسلة جبال الألب، التي تمتد في الجنوب وتغطي حوالي ثلثي مساحة البلاد ويبلغ ارتفاع أعلى قممها "بونتا دوفور" 4638م. ثم هناك سلسلة جبال جورا والتي تمتد على شكل هلال في غرب وشمال البلاد، وتمثل الحد الفاصل بين سويسرا وفرنسا وتغطي نحو 12٪ من المساحة الكلية،'],
  'id': '3'},
 {'answer': ['', 'ألمانيا من الشمال وإيطاليا'], 'id': '4'},
 {'answer': ['ليختنشتاين', 'والنمسا وإمارة ليختنشتاين', 'ليختنشتاين من الشرق'],
  'id': '5'}]

In [ ]:
evaluate(GT, prediction)

context:  تقع سويسرا في قلب القارة الأوروبية وتحيط بها خمس دول، وهي ألمانيا من الشمال وإيطاليا من الجنوب والنمسا وإمارة ليختنشتاين من الشرق وفرنسا من الغرب، وليست لها منافذ بحرية وتبلغ مساحتها حوالي 41300 كيلومترا مربعا. وتتكون سويسرا من ثلاث مناطق جغرافية وهي: سلسلة جبال الألب، التي تمتد في الجنوب وتغطي حوالي ثلثي مساحة البلاد ويبلغ ارتفاع أعلى قممها "بونتا دوفور" 4638م. ثم هناك سلسلة جبال جورا والتي تمتد على شكل هلال في غرب وشمال البلاد، وتمثل الحد الفاصل بين سويسرا وفرنسا وتغطي نحو 12٪ من المساحة الكلية، ويبلغ ارتفاع أعلى قممها "كريت دو لا نيج" 1718 م، وبين هاتين المجموعتين من السلاسل الجبلية، تمتد منطقة الهضبة السهلية التي تضم معظم المدن والقرى السويسرية.
question:  كم دولة على الحدود مع سويسرا

 gt: خمس دول
prediction: سويسرا وفرنسا    --score:  False
prediction: وفرنسا    --score:  False
prediction: سويسرا وفرنسا وتغطي نحو 12٪ من المساحة الكلية، ويبلغ ارتفاع أعلى قممها "كريت دو لا    --score:  False
prediction: سويسرا وفرنسا    --score:  0
prediction: وفرنسا    --score:  0
predic

{'exact_match': 50.0, 'f1': 69.04761904761904}